In [ ]:
!pip install komm

In [ ]:
!pip install --force-reinstall tsai

In [ ]:
!pip install --force-reinstall -v "numpy == 1.23.1"



In [ ]:
import datagen
import numpy as np

In [ ]:

resultRF, resultIQ = datagen.genSineFile(Ns=1000)
np.save("Valid/SineWaveOG.npy", resultRF)
np.save("Valid/SineWaveIQ.npy", resultIQ)
resultRF, resultIQ = datagen.genNoiseFile(Ns=1000)
np.save("Valid/NoiseOG.npy", resultRF)
np.save("Valid/NoiseIQ.npy", resultIQ)

resultRF, resultIQ = datagen.genSineFile(Ns=1000)
np.save("TrainTest/SineWaveOG.npy", resultRF)
np.save("TrainTest/SineWaveIQ.npy", resultIQ)
resultRF, resultIQ = datagen.genNoiseFile(Ns=1000)
np.save("TrainTest/NoiseOG.npy", resultRF)
np.save("TrainTest/NoiseIQ.npy", resultIQ)

RANDOM ASS STUFF

In [ ]:
!pip3 install Cython

TSAI Models

In [ ]:

arch_names =  ['InceptionTimePlus', 'InCoordTime', 'XCoordTime', 'InceptionTimeXLPlus', 'MultiInceptionTimePlus', 'MiniRocketPlus',
              'InceptionRocketPlus', 'gMLP', 'MultiInputNet', 'OmniScaleCNN', 'RNNPlus', 'LSTMPlus', 'GRUPlus','MRNN_FCN', 'MLSTM_FCN', 'MGRU_FCN', 'ResCNN',
              'ResNetPlus',  'MultiTSTPlus', 'TSiTPlus','TSTabFusionTransformer', 'xresnet1d152plus','xresnet1d50_deeperplus',  'XceptionTimePlus', 'mWDN',
              'TSSequencerPlus', "PatchTST", "RNNAttention", "LSTMAttention", "GRUAttention"]

No CUDA - This is slow

In [ ]:
from tsai import all

splits = get_splits(y,1,valid_size = 0.33,
                    check_splits = True, shuffle = True, balance = True,stratify=True)

tfms = [None, TSClassification()]
batch_tfms = TSStandardize()
model = TSClassifier(X, y, splits=splits, path='models', arch="LSTMAttention", tfms=tfms, batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph())
model.fit_one_cycle(10, 0.0005)
model.export("LSTMAttentionIQ.pkl")

In [ ]:
import datagen

import importlib
import numpy as np
import matplotlib.pyplot as plt
import komm

importlib.reload(datagen)

In [ ]:
X_true = np.load('Valid/SineWaveIQ.npy')
X_false = np.load('Valid/NoiseIQ.npy')
y_true = np.ones(shape=(X_true.shape[0]))
y_false = np.zeros(shape=(X_false.shape[0]))
X_valid = np.concatenate((X_true,X_false), axis=0).swapaxes(1,2)
y_valid = np.concatenate((y_true,y_false),axis=0)

X_true = np.load('TrainTest/SineWaveIQ.npy')
X_false = np.load('TrainTest/NoiseIQ.npy')
y_true = np.ones(shape=(X_true.shape[0]))
y_false = np.zeros(shape=(X_false.shape[0]))
X = np.concatenate((X_true,X_false), axis=0).swapaxes(1,2)
y = np.concatenate((y_true,y_false),axis=0)


In [ ]:
splitter = RandomSplitter (valid_pct=0.2, seed=None)


device= torch.device("cuda") is the key here


In [ ]:
from tsai.basics import *

splits = get_splits(y,1,valid_size = 0.33,
                    check_splits = True, shuffle = True, balance = True,stratify=True)

tfms = [None, TSClassification()]
batch_tfms = TSStandardize()
model = TSClassifier(X, y, splits=splits, path='models', arch="MiniRocketPlus", tfms=tfms,
                     device= torch.device("cuda"), batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph())
model.fit_one_cycle(10, 0.0005)
model.export("MiniRocketPlus.pkl")

probas, _, __ = model.get_X_preds(X_valid)
np.save("MiniRocketPlus", probas)


In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
from tsai.basics import *

# models_of_interest =  [  'TSSequencerPlus', 'MiniRocketPlus']
# models_of_interest =  ['InceptionTimePlus', 'XceptionTimePlus', 'LSTMAttention', 'TSSequencerPlus', 'MiniRocketPlus']
# models_of_interest =  ['TSSequencerPlus', 'MiniRocketPlus']
models_of_interest =  ['LSTMAttention']
num_epochs = [5,10,10,10,10]

for i in range(len(models_of_interest)):
  model_name = models_of_interest[i]
  epochs = num_epochs[i]

  splits = get_splits(y,1,valid_size = 0.33,
                      check_splits = True, shuffle = True, balance = True,stratify=True)

  tfms = [None, TSClassification()]
  batch_tfms = TSStandardize()
  model = TSClassifier(X, y, splits=splits, path='models', arch=model_name, tfms=tfms,
                      device= torch.device("cuda"), batch_tfms=batch_tfms, bs=[16,32],metrics=accuracy, cbs=ShowGraph())
  model.fit_one_cycle(epochs, 0.0005)
  model.export((model_name + ".pkl"))
  probas, _, __ = model.get_X_preds(X_valid)
  np.save(model_name, probas)


In [ ]:
from tsai.basics import *

model = load_learner("models/LSTMAttention.pkl", cpu=False)
probas, _, __ = model.get_X_preds(X_valid)
np.save("LSTMAttention", probas)

In [ ]:
!nvidia-smi -L

In [ ]:
learn = load_learner("models/TSSequencerPlus.pkl", cpu=False)

probas, _, __ = learn.get_X_preds(X_valid)

In [ ]:
import random as rm

In [ ]:

threshold = 0.88

index = 35000
total = 0
for i in range(35000):
  sample = probas[index][1] > threshold
  total = total + sample
  index = index + 1
PFA = total / 35000
print("PFA:", PFA)


index = 0
SNRs = np.arange(-30,5)
PDs = np.zeros(len(SNRs))
for SNR in SNRs:
  total = 0
  for i in range(1000):
    sample = probas[index][1] > threshold
    total = total + sample
    index = index + 1
  PDs[int(index/1000 - 1)] = total / 1000

plt.plot(SNRs, PDs)
TSSequencerPlus =PDs
np.save("TSSequencerPlus", TSSequencerPlus)

Energy Detector

RF

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 100
Fs = 2048000
Thresh=136
for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=2048000,Ns = Ns)
    waveNoisy = datagen.genNoise(Ns=Ns,scaled=False)


    # plt.plot(waveOriginal)
    # plt.show()
    SNRLin = pow(10, SNR/10)

    SigPower = SNRLin
    waveOriginal = waveOriginal * SNRLin
    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)

    # plt.plot(waveOriginal)
    # plt.show()

    if np.sum(np.power(np.abs(waveOriginal),2)) > Thresh:
      PD = PD + 1


    if np.sum(np.power(np.abs(waveNoisy),2)) > Thresh:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()

    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsED100 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 500
Fs = 2048000
Thresh=576
for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=2048000,Ns = Ns)
    waveNoisy = datagen.genNoise(Ns=Ns,scaled=False)


    # plt.plot(waveOriginal)
    # plt.show()
    SNRLin = pow(10, SNR/10)

    SigPower = SNRLin
    waveOriginal = waveOriginal * SNRLin
    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)

    # plt.plot(waveOriginal)
    # plt.show()

    if np.sum(np.power(np.abs(waveOriginal),2)) > Thresh:
      PD = PD + 1


    if np.sum(np.power(np.abs(waveNoisy),2)) > Thresh:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()

    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  # print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsED500 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 1000
Fs = 2048000
Thresh=1107


for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=2048000,Ns = Ns)
    waveNoisy = datagen.genNoise(Ns=Ns,scaled=False)


    # plt.plot(waveOriginal)
    # plt.show()
    SNRLin = pow(10, SNR/10)

    SigPower = SNRLin
    waveOriginal = waveOriginal * SNRLin
    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)

    # plt.plot(waveOriginal)
    # plt.show()

    if np.sum(np.power(np.abs(waveOriginal),2)) > Thresh:
      PD = PD + 1


    if np.sum(np.power(np.abs(waveNoisy),2)) > Thresh:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()

    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsED1000 = PDs

In [ ]:

np.save("PDED1000RF.npy",PDsED1000)
np.save("PDED500RF.npy",PDsED500)
np.save("PDED100RF.npy",PDsED100)
plt.plot(PDsED1000)
plt.plot(PDsED500)
plt.plot(PDsED100)


IQ

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 100
Thresh = 9.5
Fs = 2048000


for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns+100)
    noisy = datagen.genNoise(Ns=Ns + 100,scaled=False)

    # plt.plot(waveOriginal)
    # plt.show()
    SNRLin = pow(10, SNR/10)

    SigPower = SNRLin
    waveOriginal = waveOriginal * SNRLin
    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)

    waveOriginalIQ = datagen.convertToIQ(waveOriginal,fReciever = 75000,fs =Fs )
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]

    noisyIQ = datagen.convertToIQ(noisy,fReciever = 75000,fs =Fs )
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]

    # print(waveOriginalIQ.shape)
    # plt.plot(waveOriginal)
    # plt.show()

    if np.sum(np.power(np.abs(waveOriginalIQ),2)) > Thresh:
      PD = PD + 1


    if np.sum(np.power(np.abs(noisyIQ),2)) > Thresh:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()

    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsED100 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 500
Thresh=31
Fs = 2048000


for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns+100)
    noisy = datagen.genNoise(Ns=Ns + 100,scaled=False)

    # plt.plot(waveOriginal)
    # plt.show()
    SNRLin = pow(10, SNR/10)

    SigPower = SNRLin
    waveOriginal = waveOriginal * SNRLin
    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)

    waveOriginalIQ = datagen.convertToIQ(waveOriginal,fReciever = 75000,fs =Fs )
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]

    noisyIQ = datagen.convertToIQ(noisy,fReciever = 75000,fs =Fs )
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]

    # print(waveOriginalIQ.shape)
    # plt.plot(waveOriginal)
    # plt.show()

    if np.sum(np.power(np.abs(waveOriginalIQ),2)) > Thresh:
      PD = PD + 1


    if np.sum(np.power(np.abs(noisyIQ),2)) > Thresh:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()

    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsED500 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 1000
Thresh=55.2
Fs = 2048000


for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns+100)
    noisy = datagen.genNoise(Ns=Ns + 100,scaled=False)

    # plt.plot(waveOriginal)
    # plt.show()
    SNRLin = pow(10, SNR/10)

    SigPower = SNRLin
    waveOriginal = waveOriginal * SNRLin
    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)

    waveOriginalIQ = datagen.convertToIQ(waveOriginal,fReciever = 75000,fs =Fs )
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]

    noisyIQ = datagen.convertToIQ(noisy,fReciever = 75000,fs =Fs )
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]

    # print(waveOriginalIQ.shape)
    # plt.plot(waveOriginal)
    # plt.show()

    if np.sum(np.power(np.abs(waveOriginalIQ),2)) > Thresh:
      PD = PD + 1


    if np.sum(np.power(np.abs(noisyIQ),2)) > Thresh:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()

    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsED1000 = PDs

In [ ]:

np.save("PDED1000IQ.npy",PDsED1000)
np.save("PDED500IQ.npy",PDsED500)
np.save("PDED100IQ.npy",PDsED100)
plt.plot(PDsED1000)
plt.plot(PDsED500)
plt.plot(PDsED100)


In [ ]:
!pip install komm

In [ ]:
import datagen
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
for i in range(30):

  SNRs = np.arange(-30, 5)
  PDs = np.zeros((35,1))
  PFA = 0
  Iter = 1000
  Ns = 100
  Fs = 2048000
  Thresh=11.75
  for SNR in SNRs:
    PD = 0
    for i in range(Iter):
      waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns)
      noise = datagen.genNoise(Ns=Ns,scaled=True)


      SNRRandRange = SNR - 0.5 + rm.random()
      SNRLin = pow(10, SNR/10)

      waveOriginal = waveOriginal * SNRLin
      SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
      # plt.plot(waveOriginal)
      # plt.show()

      awgn = komm.AWGNChannel(SNRLin,SigPower)
      waveOriginal= awgn(waveOriginal)
      waveOriginal = waveOriginal / np.max(np.abs(waveOriginal))

      # plt.plot(waveOriginal)
      # plt.show()

      fourier = np.fft.fft(waveOriginal)
      # plt.plot(fourier)
      # plt.show()
      if np.max(np.abs(fourier)) > Thresh:
        PD = PD + 1

      fourier = np.fft.fft(noise)
      if np.max(np.abs(fourier)) > Thresh:
        PFA = PFA + 1

    PD = PD /Iter
    # print(PD)
    PDs[SNR + 30] = PD

  PFA = PFA / (len(SNRs) * Iter)
  print(PFA)
  PDsFFT100 = PDs

Fisher Statistic Thresholding

RF

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 100
Fs = 2048000
Thresh=11.7
for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns)
    noise = datagen.genNoise(Ns=Ns,scaled=True)


    SNRRandRange = SNR - 0.5 + rm.random()
    SNRLin = pow(10, SNR/10)

    waveOriginal = waveOriginal * SNRLin
    SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
    # plt.plot(waveOriginal)
    # plt.show()

    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)
    waveOriginal = waveOriginal / np.max(np.abs(waveOriginal))

    # plt.plot(waveOriginal)
    # plt.show()

    fourier = np.fft.fft(waveOriginal)
    # plt.plot(fourier)
    # plt.show()
    if np.max(np.abs(fourier)) > Thresh:
      PD = PD + 1

    fourier = np.fft.fft(noise)
    if np.max(np.abs(fourier)) > Thresh:
      PFA = PFA + 1

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsFFT100 = PDs

In [ ]:
np.save("PDsFFT100IQ",PDsFFT100IQ)

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 500
Fs = 2048000
Thresh=24

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns)
    noise = datagen.genNoise(Ns=Ns,scaled=True)


    SNRRandRange = SNR - 0.5 + rm.random()
    SNRLin = pow(10, SNR/10)

    waveOriginal = waveOriginal * SNRLin
    SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
    # plt.plot(waveOriginal)
    # plt.show()

    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)
    waveOriginal = waveOriginal / np.max(np.abs(waveOriginal))

    # plt.plot(waveOriginal)
    # plt.show()

    fourier = np.fft.fft(waveOriginal)
    # plt.plot(fourier)
    # plt.show()
    if np.max(np.abs(fourier)) > Thresh:
      PD = PD + 1

    fourier = np.fft.fft(noise)
    if np.max(np.abs(fourier)) > Thresh:
      PFA = PFA + 1

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsFFT500 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 1000
Thresh=32.45
Fs = 2048000

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns)
    noise = datagen.genNoise(Ns=Ns,scaled=True)


    SNRRandRange = SNR - 0.5 + rm.random()
    SNRLin = pow(10, SNR/10)

    waveOriginal = waveOriginal * SNRLin
    SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
    # plt.plot(waveOriginal)
    # plt.show()

    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)
    waveOriginal = waveOriginal / np.max(np.abs(waveOriginal))

    # plt.plot(waveOriginal)
    # plt.show()

    fourier = np.fft.fft(waveOriginal)
    # plt.plot(fourier)
    # plt.show()
    if np.max(np.abs(fourier)) > Thresh:
      PD = PD + 1

    fourier = np.fft.fft(noise)
    if np.max(np.abs(fourier)) > Thresh:
      PFA = PFA + 1

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsFFT1000 = PDs

In [ ]:

np.save("PDsFFT1000RF.npy",PDsFFT1000)

In [ ]:
np.save("PDsFFT100RF.npy",PDsFFT100)
np.save("PDsFFT500RF.npy",PDsFFT500)
np.save("PDsFFT1000RF.npy",PDsFFT1000)
plt.plot(PDsFFT100)
plt.plot(PDsFFT500)
plt.plot(PDsFFT1000)
plt.show()

IQ

In [ ]:
# SNRs = np.arange(-30, 5)
# PDs = np.zeros((35,1))
# PFA = 0
# Iter = 10
# Ns = 100
# Fs = 2048000
# Thresh = 70
# Omega= Ns/2 - 1

# for SNR in SNRs:
#   PD = 0
#   for i in range(Iter):
#     waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns + 100)



#     SNRRandRange = SNR - 0.5 + rm.random()
#     SNRLin = pow(10, SNR/10)
#     SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
#     NoisePower = SigPower / SNRLin
#     noise = datagen.genNoise(Ns=Ns+100,sigma = np.sqrt(NoisePower),scaled=False)

#     waveNoisy= waveOriginal + noise




#     waveOriginalIQ = datagen.convertToIQ(waveOriginal,fReciever = 75000,fs =Fs )
#     waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
#     # waveOriginalIQ = waveOriginalIQ / np.max(np.abs(waveOriginalIQ))


#     noisyIQ = datagen.convertToIQ(noise,fReciever = 75000,fs =Fs )
#     noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]
#     # noisyIQ = noisyIQ / np.max(np.abs(noisyIQ))

#     plt.plot(waveOriginal)
#     plt.show()

#     fourier = np.fft.fft(waveOriginalIQ)
#     I = fourier * np.conj(fourier)/(2*np.pi*Ns);
#     F = np.max(I)/np.sum(I)
#     Z = Ns*F-np.log(Ns)
#     # plt.plot(fourier)
#     # plt.show()
#     # print(Z)
#     if np.max(np.abs(Z)) > Thresh:
#       PD = PD + 1

#     fourier = np.fft.fft(noisyIQ)
#     I = fourier * np.conj(fourier)/(2*np.pi*Ns);
#     # print(I.shape)
#     F = max(I)/sum(I);
#     Z = Ns*F-np.log(Ns)
#     # print(Z)
#     if np.max(np.abs(Z)) > Thresh:
#       PFA = PFA + 1

#   PD = PD /Iter
#   print("SNR:",SNR,PD)
#   PDs[SNR + 30] = PD

# PFA = PFA / (len(SNRs) * Iter)
# print("PFA:", PFA)
# PDsFFT100IQ = PDs

In [ ]:
!pip install komm

In [ ]:
import numpy as np
import komm
import random as rm
import datagen


In [ ]:
0.01
0.009
0.012
0.019
0.012
0.015
0.019
0.014
0.015
0.014
0.015
0.022
0.03
0.03
0.029
0.041
0.047
0.079
0.087
0.154
0.186
0.235
0.353
0.451
0.553
0.656
0.772
0.856
0.927
0.977
0.99
0.998
1.0
1.0
1.0

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 100
Fs = 2048000
Thresh=7.95

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns + 100)
    noisy = datagen.genNoise(Ns=Ns + 100,scaled=True)


    SNRRandRange = SNR - 0.5 + rm.random()
    SNRLin = pow(10, SNR/10)

    waveOriginal = waveOriginal * SNRLin
    SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
    # plt.plot(waveOriginal)
    # plt.show()

    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)
    waveOriginal = waveOriginal / np.max(np.abs(waveOriginal))



    waveOriginalIQ = datagen.convertToIQ(waveOriginal,fReciever = 75000,fs =Fs )
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    # waveOriginalIQ = waveOriginalIQ / np.max(np.abs(waveOriginalIQ))


    noisyIQ = datagen.convertToIQ(noisy,fReciever = 75000,fs =Fs )
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]
    # noisyIQ = noisyIQ / np.max(np.abs(noisyIQ))

    # plt.plot(waveOriginal)
    # plt.show()

    fourier = np.fft.fft(waveOriginalIQ)
    # plt.plot(fourier)
    # plt.show()
    if np.max(np.abs(fourier)) > Thresh:
      PD = PD + 1

    fourier = np.fft.fft(noisyIQ)
    if np.max(np.abs(fourier)) > Thresh:
      PFA = PFA + 1

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print("PFA:", PFA)
PDsFFT100 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 500
Fs = 2048000
Thresh= 147.5

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns + 100)
    noisy = datagen.genNoise(Ns=Ns + 100,scaled=True)


    SNRRandRange = SNR - 0.5 + rm.random()
    SNRLin = pow(10, SNR/10)

    waveOriginal = waveOriginal * SNRLin
    SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
    # plt.plot(waveOriginal)
    # plt.show()

    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)
    waveOriginal = waveOriginal / np.max(np.abs(waveOriginal))



    waveOriginalIQ = datagen.convertToIQ(waveOriginal,fReciever = 75000,fs =Fs )
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    waveOriginalIQ = waveOriginalIQ / np.max(np.abs(waveOriginalIQ))


    noisyIQ = datagen.convertToIQ(noisy,fReciever = 75000,fs =Fs )
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]
    noisyIQ = noisyIQ / np.max(np.abs(noisyIQ))

    # plt.plot(waveOriginal)
    # plt.show()

    fourier = np.fft.fft(waveOriginalIQ)
    # plt.plot(fourier)
    # plt.show()
    if np.max(np.abs(fourier)) > Thresh:
      PD = PD + 1

    fourier = np.fft.fft(noisyIQ)
    if np.max(np.abs(fourier)) > Thresh:
      PFA = PFA + 1

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print("PFA:", PFA)
PDsFFT500 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
PFA = 0
Iter = 1000
Ns = 1000
Fs = 2048000
Thresh= 206

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal = datagen.genSineWave(fc = 74000 + rm.random() * 2000,fs=Fs,Ns = Ns + 100)
    noisy = datagen.genNoise(Ns=Ns + 100,scaled=True)


    SNRRandRange = SNR - 0.5 + rm.random()
    SNRLin = pow(10, SNR/10)

    waveOriginal = waveOriginal * SNRLin
    SigPower = np.linalg.norm(waveOriginal)**2 / waveOriginal.size
    # plt.plot(waveOriginal)
    # plt.show()

    awgn = komm.AWGNChannel(SNRLin,SigPower)
    waveOriginal= awgn(waveOriginal)
    waveOriginal = waveOriginal / np.max(np.abs(waveOriginal))



    waveOriginalIQ = datagen.convertToIQ(waveOriginal,fReciever = 75000,fs =Fs )
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    waveOriginalIQ = waveOriginalIQ / np.max(np.abs(waveOriginalIQ))


    noisyIQ = datagen.convertToIQ(noisy,fReciever = 75000,fs =Fs )
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]
    noisyIQ = noisyIQ / np.max(np.abs(noisyIQ))

    # plt.plot(waveOriginal)
    # plt.show()

    fourier = np.fft.fft(waveOriginalIQ)
    # plt.plot(fourier)
    # plt.show()
    if np.max(np.abs(fourier)) > Thresh:
      PD = PD + 1

    fourier = np.fft.fft(noisyIQ)
    if np.max(np.abs(fourier)) > Thresh:
      PFA = PFA + 1

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print("PFA:", PFA)
PDsFFT1000 = PDs

In [ ]:
np.save("PDsFFT100IQ.npy",PDsFFT100)
np.save("PDsFFT500IQ.npy",PDsFFT500)
np.save("PDsFFT1000IQ.npy",PDsFFT1000)
plt.plot(PDsFFT100)
plt.plot(PDsFFT500)
plt.plot(PDsFFT1000)
plt.show()

In [ ]:
IQ1 = np.load("PDsFFT100IQ.npy")
IQ5 = np.load("PDsFFT500IQ.npy")
IQ10 = np.load("PDsFFT1000IQ.npy")
RF1 = np.load("PDsFFT100RF.npy")
RF5 = np.load("PDsFFT500RF.npy")
RF10 = np.load("PDsFFT1000RF.npy")

In [ ]:
plt.plot(RF1,label="RF1")
plt.plot(RF5,label="RF5")
plt.plot(RF10,label="RF10")
plt.plot(IQ1,label="IQ1")
plt.plot(IQ5,label="IQ5")
plt.plot(IQ10,label="IQ10")
plt.legend()

Matched Filtering


In [ ]:
!pip install komm

In [ ]:
import datagen
import importlib
import numpy as np
import matplotlib.pyplot as plt

importlib.reload(datagen)

In [ ]:

waveOriginal, waveNoisy,SNR = datagen.genSineWavePaired(0,Ns = 500, scaled=False)

In [ ]:
plt.plot(waveOriginal)

In [ ]:
from matplotlib import pyplot as plt
from scipy import special as sp

def invQfunc(x):
    return np.sqrt(2)*sp.erfinv(1-2*x)
# plt.plot(noisy)

In [ ]:
SNRTrue = -30
SNRLin = pow(10, SNRTrue/10)
sigma = np.sqrt(1/SNRLin/2);
eta = sigma/np.sqrt(2*Ns)*invQfunc(DesiredPfa);

In [ ]:
print(eta)

In [ ]:
print(sigma)

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 100
scale = 1.34
for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns, scaled=True)


    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(2*Ns)*invQfunc(DesiredPfa);
    eta = eta * scale



    # print(eta)
    filtered = np.convolve(waveOriginal,waveNoisy) / Ns
    # print(eta)
    if np.max(np.abs(filtered)) > eta:
      PD = PD + 1

    noisy = datagen.genNoise(Ns = Ns, scaled=False,sigma=sigma)
    filtered = np.convolve(waveOriginal,noisy) / Ns
    # print(eta)
    if np.max(np.abs(filtered)) > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsMF100 = PDs

Baseband Matched Filtering


In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 100
scale = 1

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns, scaled=False)


    # print(waveOriginal.shape)
    # print(waveNoisy.shape)
    # print(waveNoisy.shape)


    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(2*Ns)*invQfunc(DesiredPfa);
    eta = eta * scale

    Total_power = np.sum(np.abs(waveNoisy))**2 / waveNoisy.size
    ## WEIRD THING HERE
    Noise_power = Total_power / (1 + SNRLin)
    Signal_power = Total_power - Noise_power


    # print(SNRLin)
    # print(eta)
    filtered = np.matmul(waveOriginal,waveNoisy)
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PD = PD + 1

    noisy = waveNoisy - waveOriginal
    filtered = np.matmul(waveOriginal,noisy)
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # # plt.plot(filtered)
    # # plt.show()

    # print(Total_power)
    # print("Calculated",Signal_power)
    # print("Real", np.linalg.norm(waveOriginal)**2 / waveOriginal.size)
    # print("Calculated",Noise_power)
    # print("Real", np.linalg.norm(noisy)**2 / noisy.size)
  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsMF100 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 500
scale = 1

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns, scaled=False)


    # print(waveOriginal.shape)
    # print(waveNoisy.shape)
    # print(waveNoisy.shape)


    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(2*Ns)*invQfunc(DesiredPfa);
    eta = eta * scale

    Total_power = np.sum(np.abs(waveNoisy))**2 / waveNoisy.size
    ## WEIRD THING HERE
    Noise_power = Total_power / (1 + SNRLin)
    Signal_power = Total_power - Noise_power


    # print(SNRLin)
    # print(eta)
    filtered = np.matmul(waveOriginal,waveNoisy)
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PD = PD + 1

    noisy = waveNoisy - waveOriginal
    filtered = np.matmul(waveOriginal,noisy)
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # # plt.plot(filtered)
    # # plt.show()

    # print(Total_power)
    # print("Calculated",Signal_power)
    # print("Real", np.linalg.norm(waveOriginal)**2 / waveOriginal.size)
    # print("Calculated",Noise_power)
    # print("Real", np.linalg.norm(noisy)**2 / noisy.size)
  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsMF500 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 1000
scale = 1

for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns, scaled=False)


    # print(waveOriginal.shape)
    # print(waveNoisy.shape)
    # print(waveNoisy.shape)


    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(2*Ns)*invQfunc(DesiredPfa);
    eta = eta * scale

    Total_power = np.sum(np.abs(waveNoisy))**2 / waveNoisy.size
    ## WEIRD THING HERE
    Noise_power = Total_power / (1 + SNRLin)
    Signal_power = Total_power - Noise_power


    # print(SNRLin)
    # print(eta)
    filtered = np.matmul(waveOriginal,waveNoisy)
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PD = PD + 1

    noisy = waveNoisy - waveOriginal
    filtered = np.matmul(waveOriginal,noisy)
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # # plt.plot(filtered)
    # # plt.show()

    # print(Total_power)
    # print("Calculated",Signal_power)
    # print("Real", np.linalg.norm(waveOriginal)**2 / waveOriginal.size)
    # print("Calculated",Noise_power)
    # print("Real", np.linalg.norm(noisy)**2 / noisy.size)
  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsMF1000 = PDs

In [ ]:
np.save("PDMF100RF.npy",PDsMF100)
np.save("PDMF500RF.npy",PDsMF500)
np.save("PDMF1000RF.npy",PDsMF1000)
plt.plot(PDsMF100)
plt.plot(PDsMF500)
plt.plot(PDsMF1000)
plt.show()

Matched Filtering IQ


In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 100
scale = 1.34



for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns+100, scaled=False)
    waveOriginalIQ = datagen.convertToIQ(waveOriginal)
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    adj = np.max(np.abs(waveOriginalIQ))
    waveOriginalIQ = waveOriginalIQ/adj
    waveNoisyIQ = datagen.convertToIQ(waveNoisy)
    waveNoisyIQ = waveNoisyIQ[:,0] + 1j*waveNoisyIQ[:,1]

    waveNoisyIQ = waveNoisyIQ/adj
    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(Ns)*invQfunc(DesiredPfa);


    eta = eta * scale



    # print(eta)
    filtered = np.real(np.matmul(waveOriginalIQ,np.conj(waveNoisyIQ)))
    # plt.plot(filtered)
    # plt.show()
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PD = PD + 1

    noisy = waveNoisy - waveOriginal
    noisyIQ = datagen.convertToIQ(noisy)
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]

    noisyIQ = waveNoisyIQ - waveOriginalIQ
    filtered = np.real(np.matmul(waveOriginalIQ,np.conj(noisyIQ)))
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
PDsMF100 = PDs
print(PFA)

In [ ]:

np.save("PDsMF100IQ.npy",PDsMF100)

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 500
scale = 1.35


for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns+100, scaled=False)
    waveOriginalIQ = datagen.convertToIQ(waveOriginal)
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    adj = np.max(np.abs(waveOriginalIQ))
    waveOriginalIQ = waveOriginalIQ/adj
    waveNoisyIQ = datagen.convertToIQ(waveNoisy)
    waveNoisyIQ = waveNoisyIQ[:,0] + 1j*waveNoisyIQ[:,1]

    waveNoisyIQ = waveNoisyIQ/adj
    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(Ns)*invQfunc(DesiredPfa);


    eta = eta * scale



    # print(eta)
    filtered = np.real(np.matmul(waveOriginalIQ,np.conj(waveNoisyIQ)))
    # plt.plot(filtered)
    # plt.show()
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PD = PD + 1

    noisy = waveNoisy - waveOriginal
    noisyIQ = datagen.convertToIQ(noisy)
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]

    noisyIQ = waveNoisyIQ - waveOriginalIQ
    filtered = np.real(np.matmul(waveOriginalIQ,np.conj(noisyIQ)))
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
PDsMF500 = PDs
print(PFA)

In [ ]:

np.save("PDsMF500IQ.npy",PDsMF500)

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 500
Ns = 1000
scale = 1.345


for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns+100, scaled=False)
    waveOriginalIQ = datagen.convertToIQ(waveOriginal)
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    adj = np.max(np.abs(waveOriginalIQ))
    waveOriginalIQ = waveOriginalIQ/adj
    waveNoisyIQ = datagen.convertToIQ(waveNoisy)
    waveNoisyIQ = waveNoisyIQ[:,0] + 1j*waveNoisyIQ[:,1]

    waveNoisyIQ = waveNoisyIQ/adj
    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(Ns)*invQfunc(DesiredPfa);


    eta = eta * scale



    # print(eta)
    filtered = np.real(np.matmul(waveOriginalIQ,np.conj(waveNoisyIQ)))
    # plt.plot(filtered)
    # plt.show()
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PD = PD + 1

    noisy = waveNoisy - waveOriginal
    noisyIQ = datagen.convertToIQ(noisy)
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]

    noisyIQ = waveNoisyIQ - waveOriginalIQ
    filtered = np.real(np.matmul(waveOriginalIQ,np.conj(noisyIQ)))
    # print(eta)
    if np.sum((filtered))/ Ns > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
PDsMF1000 = PDs
print(PFA)

In [ ]:

np.save("PDsMF1000IQ.npy",PDsMF1000)

In [ ]:
plt.plot(PDs)

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 500



for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns + 100, scaled=True)
    waveOriginalIQ = datagen.convertToIQ(waveOriginal)
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    waveNoisyIQ = datagen.convertToIQ(waveNoisy)
    waveNoisyIQ = waveNoisyIQ[:,0] + 1j*waveNoisyIQ[:,1]

    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(2*Ns)*invQfunc(DesiredPfa);
    scale = 0.68

    eta = eta * scale



    # print(eta)
    filtered = np.convolve(waveOriginalIQ,waveNoisyIQ) / Ns
    # plt.plot(filtered)
    # plt.show()
    # print(eta)
    if np.max(np.abs(filtered)) > eta:
      PD = PD + 1

    noisy = datagen.genNoise(Ns = Ns + 100, scaled=False,sigma=sigma)
    noisyIQ = datagen.convertToIQ(noisy)
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]
    filtered = np.convolve(waveOriginalIQ,noisyIQ) / Ns
    # print(filtered.shape)
    if np.max(np.abs(filtered)) > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsMF500 = PDs

In [ ]:
SNRs = np.arange(-30, 5)
PDs = np.zeros((35,1))
DesiredPfa = 0.01
PFA = 0
Iter = 1000
Ns = 1000



for SNR in SNRs:
  PD = 0
  for i in range(Iter):
    waveOriginal, waveNoisy, SNRTrue = datagen.genSineWavePaired(SNR,Ns = Ns + 100, scaled=True)
    waveOriginalIQ = datagen.convertToIQ(waveOriginal)
    waveOriginalIQ = waveOriginalIQ[:,0] + 1j*waveOriginalIQ[:,1]
    waveNoisyIQ = datagen.convertToIQ(waveNoisy)
    waveNoisyIQ = waveNoisyIQ[:,0] + 1j*waveNoisyIQ[:,1]

    SNRLin = pow(10, SNRTrue/10)
    sigma = np.sqrt(1/SNRLin/2);
    eta = sigma/np.sqrt(2*Ns)*invQfunc(DesiredPfa);
    scale = 0.69
    eta = eta * scale



    # print(eta)
    filtered = np.convolve(waveOriginalIQ,waveNoisyIQ) / Ns
    # plt.plot(filtered)
    # plt.show()
    # print(eta)
    if np.max(np.abs(filtered)) > eta:
      PD = PD + 1

    noisy = datagen.genNoise(Ns = Ns + 100, scaled=False,sigma=sigma)
    noisyIQ = datagen.convertToIQ(noisy)
    noisyIQ = noisyIQ[:,0] + 1j*noisyIQ[:,1]
    filtered = np.convolve(waveOriginalIQ,noisyIQ) / Ns
    # print(eta)
    if np.max(np.abs(filtered)) > eta:
      PFA = PFA + 1
    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(noisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /Iter
  print(PD)
  PDs[SNR + 30] = PD

PFA = PFA / (len(SNRs) * Iter)
print(PFA)
PDsMF1000 = PDs

In [ ]:
IQ1000 = np.load("Results/PDMF1000IQ.npy")
RF1000 = np.load("Results/PDMF1000RF.npy")
IQ500 = np.load("Results/PDMF500IQ.npy")
RF500 = np.load("Results/PDMF500RF.npy")
IQ100 = np.load("Results/PDMF100IQ.npy")
RF100 = np.load("Results/PDMF100RF.npy")

plt.plot(IQ1000)
plt.plot(RF1000)
plt.plot(IQ500)
plt.plot(RF500)
plt.plot(IQ100)
plt.plot(RF100)
plt.show()

In [ ]:
SNRs = np.arange(-30, 5)
PDsConv = np.zeros((35,1))
for SNR in SNRs:
  PD = 0
  for i in range(5000):
    waveOriginal, waveNoisy = datagen.genSineWavePaired(SNR,Ns = 500, scaled=True)

    filtered = np.convolve(waveOriginal,waveNoisy)
    if np.max(np.abs(filtered)) > threshold:
      PD = PD + 1

    # plt.plot(waveNoisy)
    # plt.show()
    # plt.plot(waveOriginal)
    # plt.show()
    # plt.plot(filtered)
    # plt.show()

  PD = PD /5000
  print(PD)
  PDsConv[SNR + 30] = PD

In [ ]:
plt.plot(PDs)
plt.plot(PDsConv)
plt.show()

In [ ]:
preds = np.zeros((120000,1))
for i in range(len(preds)):
  sample = X_valid[i].swapaxes(0,1)
  # print(sample.shape)
  fourier = np.fft.fft(sample[:,0])
  # plt.plot(fourier)
  # plt.show()
  # print(fourier.shape)
  preds[i] = np.max(np.abs(fourier))

threshold = 23.3

index = 60000
total = 0
for i in range(60000):
  sample = preds[index] > threshold
  total = total + sample
  index = index + 1
PFA = total / 60000
print("PFA:", PFA)


index = 0
SNRs = np.arange(-25,5)
PDs = np.zeros(len(SNRs))
for SNR in SNRs:
  total = 0
  for i in range(2000):
    sample = preds[index] > threshold
    total = total + sample
    index = index + 1
  PDs[int(index/2000 - 1)] = total / 2000

plt.plot(SNRs, PDs)
PDFFTThreshold =PDs
np.save("PDsMCNN", PDsMCNN)

In [ ]:

plt.plot(waveOriginal)
plt.plot(waveNoisy)

In [ ]:
import datagen
import numpy as np

In [ ]:

resultOG, resultIQ = datagen.genSineFile()

np.save("TrainTest/SineWaveOG.npy", resultOG)
np.save("TrainTest/SineWaveIQ.npy", resultIQ)
resultOG, resultIQ = datagen.genNoiseFile()
np.save("TrainTest/NoiseOG.npy", resultOG)
np.save("TrainTest/NoiseIQ.npy", resultIQ)

In [ ]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, stratify = y)

In [ ]:



# resultOG, resultIQ = datagen.genSineFile()

# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, stratify = y)

In [ ]:
plt.plot(preds)

In [ ]:
from sktime.classification.deep_learning.lstmfcn import LSTMFCNClassifier
from sktime.classification.deep_learning.macnn import MACNNClassifier

In [ ]:
network = MACNNClassifier(n_epochs=5, verbose=True)
network.fit(X_train, y_train)
network.score(X_test, y_test)

In [ ]:
preds = network.predict_proba(X_valid)

In [ ]:
plt.plot(preds)


In [ ]:
threshold = 0.691

index = 60000
total = 0
for i in range(60000):
  sample = preds[index][1] > threshold
  total = total + sample
  index = index + 1
PFA = total / 30000
print("PFA:", PFA)


index = 0
PDs = np.zeros(30)
SNRs = np.arange(-25,5)
for SNR in SNRs:
  total = 0
  for i in range(2000):
    sample = preds[index][1] > threshold
    total = total + sample
    index = index + 1
  PDs[int(index/2000 - 1)] = total / 2000

plt.plot(SNRs, PDs)
PDsMACNN = PDs
np.save("PDsMACNN", PDsMACNN)

In [ ]:
network = LSTMFCNClassifier(n_epochs=20, verbose=True)
network.fit(X_train, y_train)
network.score(X_test, y_test)

In [ ]:
preds = network.predict_proba(X_valid)

In [ ]:
threshold = 0.912

index = 60000
total = 0
for i in range(60000):
  sample = preds[index][1] > threshold
  total = total + sample
  index = index + 1
PFA = total / 60000
print("PFA:", PFA)


index = 0
SNRs = np.arange(-25,5)
PDs = np.zeros(len(SNRs))
for SNR in SNRs:
  total = 0
  for i in range(2000):
    sample = preds[index][1] > threshold
    total = total + sample
    index = index + 1
  PDs[int(index/2000 - 1)] = total / 2000

plt.plot(SNRs, PDs)
PDsLSTMFCN =PDs
np.save("PDsLSTMFCN", PDsLSTMFCN)

HIVECOTE V2 - SUPPOSEDLY BEST


In [ ]:
from tsai.models.MINIROCKET import *

import numpy as np
from tsai.basics import *
import sktime
import sklearn
my_setup(sktime, sklearn)



In [ ]:

from sktime.classification.hybrid._hivecote_v2 import HIVECOTEV2

In [ ]:
print(X_train.shape)

In [ ]:

model = HIVECOTEV2(
        drcif_params={
            "contract_max_n_estimators": 1,
            "n_intervals": 2,
            "att_subsample_size": 2,
        },
        arsenal_params={"num_kernels": 5, "contract_max_n_estimators": 1},
        tde_params={
            "contract_max_n_parameter_samples": 1,
            "max_ensemble_size": 1,
            "randomly_selected_params": 1,
        },
        time_limit_in_minutes=0.25,
        random_state=0,
    )

In [ ]:

model.fit(X_train, np.array(y_train, dtype=int))


In [ ]:

preds = model.predict_proba(X_valid)


In [ ]:
threshold = 0.691

index = 60000
total = 0
for i in range(60000):
  sample = preds[index][1] > threshold
  total = total + sample
  index = index + 1
PFA = total / 30000
print("PFA:", PFA)


index = 0
PDs = np.zeros(30)
SNRs = np.arange(-25,5)
for SNR in SNRs:
  total = 0
  for i in range(2000):
    sample = preds[index][1] > threshold
    total = total + sample
    index = index + 1
  PDs[int(index/2000 - 1)] = total / 2000

plt.plot(SNRs, PDs)
PDsMACNN = PDs
np.save("PDsMACNN", PDsMACNN)

In [ ]:
!pip install numpy==1.23.1

In [ ]:
model.fit(X_train,y_train)

In [ ]:
preds = model.predict(X_valid.swapaxes(1,2))

In [ ]:
threshold = 0.78

index = 30000
total = 0
for i in range(30000):
  sample = preds[index] > threshold
  total = total + sample
  index = index + 1
PFA = total / 30000
print("PFA:", PFA)


index = 0
SNRs = np.arange(-25,5)
PDs = np.zeros(len(SNRs))
for SNR in SNRs:
  total = 0
  for i in range(1000):
    sample = preds[index] > threshold
    total = total + sample
    index = index + 1
  PDs[int(index/1000 - 1)] = total / 1000

plt.plot(SNRs, PDs)
PDsMCNN =PDs
np.save("PDsMCNN", PDsMCNN)

In [ ]:
preds = np.zeros((120000,1))
for i in range(len(preds)):
  sample = X_valid[i].swapaxes(0,1)
  # print(sample.shape)
  fourier = np.fft.fft(sample[:,0])
  # plt.plot(fourier)
  # plt.show()
  # print(fourier.shape)
  preds[i] = np.max(np.abs(fourier))

threshold = 23.3

index = 60000
total = 0
for i in range(60000):
  sample = preds[index] > threshold
  total = total + sample
  index = index + 1
PFA = total / 60000
print("PFA:", PFA)


index = 0
SNRs = np.arange(-25,5)
PDs = np.zeros(len(SNRs))
for SNR in SNRs:
  total = 0
  for i in range(2000):
    sample = preds[index] > threshold
    total = total + sample
    index = index + 1
  PDs[int(index/2000 - 1)] = total / 2000

plt.plot(SNRs, PDs)
PDFFTThreshold =PDs
np.save("PDsMCNN", PDsMCNN)

In [ ]:
plt.plot(SNRs, PDsMinirocket,label = 'Minirocket')
plt.plot(SNRs, PDsLSTMFCN, label = 'LSTM')
plt.plot(SNRs, PDsMACNN,label = 'MACNN')
plt.plot(SNRs, PDsInception, label = 'Inception')
plt.plot(SNRs, PDsMCNN, label = 'MCNN')
plt.plot(SNRs, PDFFTThreshold, label = 'FFT Threshold')
plt.legend()
plt.xlabel('SNRdB')
plt.ylabel('PD')
plt.show()

In [ ]:

plt.plot(SNRs, PDsMCNN, label = 'MCNN')
plt.plot(SNRs, PDFFTThreshold, label = 'FFT Threshold')

In [ ]:
print(X_valid.shape)

In [ ]:

  sample = X_valid[25000].swapaxes(0,1)
  # print(sample.shape)
  plt.plot(sample[:,0])
  plt.show()
  fourier = np.fft.fft(sample[:,0])
  plt.plot(fourier)
  plt.show()

In [ ]:
import numpy as np
X_true = np.load('Valid/SineWaveIQ.npy')
X_false = np.load('Valid/NoiseIQ.npy')
y_true = np.ones(shape=(X_true.shape[0]))
y_false = np.zeros(shape=(X_false.shape[0]))
X_valid = np.concatenate((X_true,X_false), axis=0).swapaxes(1,2)
y_valid = np.concatenate((y_true,y_false),axis=0)

X_true = np.load('TrainTest/SineWaveIQ.npy')
X_false = np.load('TrainTest/NoiseIQ.npy')
y_true = np.ones(shape=(X_true.shape[0]))
y_false = np.zeros(shape=(X_false.shape[0]))
X = np.concatenate((X_true,X_false), axis=0).swapaxes(1,2)
y = np.concatenate((y_true,y_false),axis=0)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.33, stratify = y)


In [ ]:

from tensorflow.keras.layers import (Input, Conv1D, Conv2D, Conv3D, SpatialDropout1D,
                                     MaxPooling1D, MaxPooling2D, MaxPooling3D, Flatten, Concatenate,
                                     Reshape,
                                     GlobalAveragePooling1D,
                                     Dropout, Dense, BatchNormalization)
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from math import floor
import numpy as np
from tensorflow.keras.utils import plot_model

def build_model():

    inputLen = 1000
    input_ = Input(shape=(inputLen,2))

    num_filters = 256
    resulting_filters = 512
    dropout = 0.1
    pooling = True


    branch_1 = Conv1D(filters=num_filters*2, kernel_size=3, dilation_rate=1, padding='same',activation = 'relu') (input_)
    branch_1 = SpatialDropout1D(dropout)(branch_1)
    branch_1= MaxPooling1D(pool_size=2, strides=2) (branch_1)
    branch_1 = Reshape((int(inputLen/2),resulting_filters,1))(branch_1)

    branch_2 = Conv1D(filters=num_filters*2, kernel_size=3, dilation_rate=2, padding='same',activation = 'relu') (input_)
    branch_2 = SpatialDropout1D(dropout)(branch_2)
    branch_2= MaxPooling1D(pool_size=2, strides=2) (branch_2)
    branch_2 = Reshape((int(inputLen/2),resulting_filters,1))(branch_2)

    branch_3 = Conv1D(filters=num_filters*2, kernel_size=3, dilation_rate=3, padding='same',activation = 'relu') (input_)
    branch_3 = SpatialDropout1D(dropout)(branch_3)
    branch_3= MaxPooling1D(pool_size=2, strides=2) (branch_3)
    branch_3 = Reshape((int(inputLen/2),resulting_filters,1))(branch_3)

    branch_4 = Conv1D(filters=num_filters*2, kernel_size=3, dilation_rate=4, padding='same',activation = 'relu') (input_)
    branch_4 = SpatialDropout1D(dropout)(branch_4)
    branch_4= MaxPooling1D(pool_size=2, strides=2) (branch_4)
    branch_4 = Reshape((int(inputLen/2),resulting_filters,1))(branch_4)

    branch_5 = Conv1D(filters=num_filters*2, kernel_size=3, dilation_rate=5, padding='same',activation = 'relu') (input_)
    branch_5 = SpatialDropout1D(dropout)(branch_5)
    branch_5= MaxPooling1D(pool_size=2, strides=2) (branch_5)
    branch_5 = Reshape((int(inputLen/2),resulting_filters,1))(branch_5)

    branch_6 = Conv1D(filters=num_filters*2, kernel_size=3, dilation_rate=6, padding='same',activation = 'relu') (input_)
    branch_6 = SpatialDropout1D(dropout)(branch_6)
    branch_6 = MaxPooling1D(pool_size=2, strides=2) (branch_6)
    branch_6 = Reshape((int(inputLen/2),resulting_filters,1))(branch_6)






    concat = Concatenate(axis=3) ([branch_1, branch_2, branch_3, branch_4, branch_5,branch_6])
    concat = Conv2D(filters = 512, kernel_size=(3,resulting_filters),activation = 'relu') (concat)
    concat = Reshape((int(inputLen/2) - 2,512))(concat)
    concat = SpatialDropout1D(0.1)(concat)
    for i in range(1,4):
        concat= Conv1D(filters=(int)(256 * (1.2**i)), kernel_size=3,padding="same",activation = 'relu')(concat)
        concat= MaxPooling1D(pool_size=2, strides=2) (concat)

    concat= Conv1D(filters=1028, kernel_size=3,padding="valid",activation = 'relu')(concat)
    concat = GlobalAveragePooling1D() (concat)
    dense1 = Flatten()(concat)
    out = Dense(1, activation = 'sigmoid') (dense1)


    model = Model(inputs=input_, outputs=out)
    weights = np.ones((1,))
    weights[0] = 3
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy', metrics.FalsePositives(), metrics.FalseNegatives()])
    return model

model = build_model()


model.summary()

history = model.fit(x=X_train.swapaxes(1,2)   , y=y_train, validation_data=(X_test.swapaxes(1,2), y_test), epochs=5, batch_size=100)


In [ ]:
preds = model.predict(X_valid.swapaxes(1,2))

In [ ]:
np.save("MCNNPreds",preds)

In [ ]:
plt.plot(preds)
plt.show()

In [ ]:
PDsTotal = np.concatenate((PDsMinirocket,PDsLSTMFCN,PDsMACNN,PDsInception,PDsMCNN))

In [ ]:
np.savetxt('PDArray.csv',PDsTotal, delimiter = ',')


In [ ]:
import pandas as pd
pd.DataFrame(PDsTotal).to_csv("PDArray.csv")
